**Image prediction (cat/dog) using CNN:**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

**Source dataset:**

In [ ]:
data_dir = tf.keras.utils.get_file(
    'cats_dogs',
    'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip',
    extract=True)
#or use: https://www.tensorflow.org/datasets/catalog/cats_vs_dogs

In [ ]:
data_dir  #Average image size: 403x358.  Found 2000 images in folder - /root/.keras/datasets/cats_dogs/cats_and_dogs_filtered/train

In [ ]:
IMAGE_SIZE = 128  #target img size
BATCH_SIZE = 16   #16 imgs as 1 batch
train_dir = data_dir + '/cats_and_dogs_filtered/train'
validation_dir = data_dir + '/cats_and_dogs_filtered/validation'

**Read from dataset:**

In [ ]:
# tf.keras.utils.image_dataset_from_directory is recommanded than ImageDataGenerator
train_ds_base = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True
)

validation_ds_base = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    labels='inferred',
    label_mode='binary',
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
class_names = tuple(train_ds_base.class_names)
train_size = train_ds_base.cardinality().numpy()
valid_size = validation_ds_base.cardinality().numpy()

class_names, train_size, valid_size   #2000/16 = 125 batches

**Apply  preprocessing (normalize) and data augmentation:**

In [ ]:
normalization_layer = layers.Rescaling(1. / 255)
augment_model = tf.keras.Sequential([
    # layers.RandomFlip("horizontal"),
    # layers.RandomRotation(0.1),
    # layers.RandomZoom(0.2),
    layers.RandomTranslation(0.1, 0.1),
    normalization_layer
])

train_ds = train_ds_base.map(lambda images, labels:
                        (augment_model(images), labels))
validation_ds = validation_ds_base.map(lambda images, labels:
                        (normalization_layer(images), labels))

**CNN architecture:**

In [ ]:
model = tf.keras.Sequential([  # = tf.keras.models.Sequential
    layers.InputLayer(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPool2D(),

    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    #layers.SpatialDropout2D(0.1),

    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    #layers.SpatialDropout2D(0.1)
])

#ANN
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1, activation= tf.keras.activations.sigmoid))
model.summary()

In [ ]:
#MBGD (BATCH_SIZE) still overrides SGD here
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.025),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history = model.fit(train_ds, validation_data=validation_ds, epochs=30,
                    steps_per_epoch=train_size, validation_steps=valid_size)

**Plot accuracy and loss:**

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower left')
plt.show()

**Test:**

In [ ]:
def predict_image(img_path):
  img = tf.keras.utils.load_img(img_path, target_size=(IMAGE_SIZE, IMAGE_SIZE))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Add batch dimension
  img_array = img_array / 255.0  # Normalize

  prediction = model.predict(img_array)

  predicted_class_index = (prediction > 0.5).astype(int)[0][0]
  predicted_class_name = class_names[predicted_class_index]
  print(f"The predicted class is: {predicted_class_name} (score:{prediction})")

In [ ]:
#Test cat image
img_path = "/root/.keras/datasets/cats_dogs/cats_and_dogs_filtered/validation/cats/cat.2036.jpg"
predict_image(img_path)

In [ ]:
#Test dog image
img_path = "/root/.keras/datasets/cats_dogs/cats_and_dogs_filtered/validation/dogs/dog.2024.jpg"
predict_image(img_path)

**System** - predict new uploaded image

In [ ]:
import ipywidgets as widgets
from IPython.display import display

upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False  # Allow only one file upload at a time
)

def on_upload_change(change):
    for name, file_info in upload_button.value.items():
        img_path = name
        # Save the uploaded file to a temporary location
        with open(img_path, 'wb') as f:
            f.write(file_info['content'])

        # Call the prediction function with the uploaded image path
        predict_image(img_path)

upload_button.observe(on_upload_change, names='value')

print("Upload an image to predict:")
display(upload_button)